# Clarity Model Evaluation - CSV Dataset (2-GPU)
Runs evaluation on `clarity_task_evaluation_dataset.csv` using 2 GPUs with FP16.

In [ ]:
%pip install transformers accelerate datasets scikit-learn -q

In [ ]:
import pandas as pd
import torch
import re
import warnings
from tqdm.notebook import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics import (
    accuracy_score, 
    precision_recall_fscore_support, 
    f1_score, 
    classification_report, 
    confusion_matrix
)

warnings.filterwarnings("ignore", message=".*attention_mask.*")

# Check GPUs
print(f"Available GPUs: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    props = torch.cuda.get_device_properties(i)
    print(f"  GPU {i}: {props.name} ({props.total_memory / 1024**3:.1f} GB)")

In [ ]:
# ==================== CONFIGURATION ====================
MODEL_NAME = "Frenzyknight/Clarity-llama-70b"  # Your model
DATA_PATH = "clarity_task_evaluation_dataset.csv"  # Input CSV
OUTPUT_PATH = "clarity_csv_predictions.csv"  # Output CSV

MAX_NEW_TOKENS = 600
TEMPERATURE = 0.1
MAX_INPUT_LENGTH = 3500
LIMIT = None  # Set to a number for testing (e.g., 10)

In [ ]:
# ==================== PROMPTS ====================
SYSTEM_PROMPT = "You are an expert political discourse analyst. Analyze political interviews step by step and classify response clarity."

USER_TEMPLATE = """Based on a segment of the interview where the interviewer asks a series of questions, classify the type of response provided by the interviewee for the following question.

### Classification Categories ###
1. Clear Reply - The information requested is explicitly stated (in the requested form)
2. Clear Non-Reply - The information requested is not given at all due to ignorance, need for clarification, or declining to answer
3. Ambivalent - The information requested is given in an incomplete way (e.g., the answer is too general, partial, implicit, dodging, or deflection)

### Full Interview Question ###
{interview_question}

### Full Interview Answer ###
{interview_answer}

### Specific Question to Classify ###
{question}

Think step by step, then provide your classification."""


def format_messages(row):
    """Format a CSV row into conversation messages."""
    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_TEMPLATE.format(
            interview_question=str(row["interview_question"]),
            interview_answer=str(row["interview_answer"]),
            question=str(row["question"])
        )}
    ]


def extract_label(response: str) -> str:
    """Extract label from model response."""
    # Look for explicit LABEL: pattern
    match = re.search(r'LABEL:\s*(Clear Reply|Clear Non-Reply|Ambivalent)', response, re.IGNORECASE)
    if match:
        label = match.group(1).strip().lower()
        if "non-reply" in label:
            return "Clear Non-Reply"
        elif "clear reply" in label:
            return "Clear Reply"
        elif "ambivalent" in label:
            return "Ambivalent"
    
    # Fallback: check last 100 chars
    last_part = response[-100:].lower() if len(response) > 100 else response.lower()
    if "clear non-reply" in last_part or "non-reply" in last_part:
        return "Clear Non-Reply"
    elif "clear reply" in last_part:
        return "Clear Reply"
    elif "ambivalent" in last_part:
        return "Ambivalent"
    
    return "PARSE_ERROR"


print("✅ Helper functions defined")

In [ ]:
# ==================== LOAD MODEL (2 GPUs) ====================
print("=" * 60)
print(f"Loading model: {MODEL_NAME}")
print("Distributing across GPUs with FP16...")
print("=" * 60)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
)

print("\n✅ Model loaded!")
if hasattr(model, 'hf_device_map'):
    devices = set(model.hf_device_map.values())
    print(f"Distributed across: {devices}")

In [ ]:
# ==================== LOAD CSV DATA ====================
print(f"Loading data from: {DATA_PATH}")
eval_df = pd.read_csv(DATA_PATH)
print(f"✅ Loaded {len(eval_df)} examples")
print(f"   Columns: {eval_df.columns.tolist()}")

if LIMIT:
    print(f"\n⚠️  Limiting to first {LIMIT} examples")
    eval_df = eval_df.head(LIMIT)

# Preview
print("\nSample row:")
print(f"  Question: {eval_df.iloc[0]['question'][:80]}...")

In [ ]:
# ==================== RUN INFERENCE ====================
results = []
total = len(eval_df)

print("\n" + "=" * 60)
print(f"Running inference on {total} examples")
print("=" * 60 + "\n")

for idx, row in tqdm(eval_df.iterrows(), total=total, desc="Evaluating"):
    # Format messages from CSV row
    messages = format_messages(row)
    
    # Apply chat template (get string), then tokenize
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    encoded = tokenizer(prompt, return_tensors="pt")
    inputs = encoded["input_ids"]
    input_len = inputs.shape[-1]
    
    # Skip if too long
    if input_len > MAX_INPUT_LENGTH:
        tqdm.write(f"[{idx + 1}/{total}] SKIPPED (too long: {input_len} tokens)")
        results.append({
            "index": row.get("index", idx),
            "question": row["question"],
            "prediction": "SKIPPED",
            "raw_output": f"Input too long: {input_len} tokens"
        })
        continue
    
    try:
        inputs = inputs.to(model.device)
        attention_mask = torch.ones_like(inputs)
        
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs,
                attention_mask=attention_mask,
                max_new_tokens=MAX_NEW_TOKENS,
                temperature=TEMPERATURE,
                do_sample=TEMPERATURE > 0,
                use_cache=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
        
        full_response = tokenizer.decode(
            outputs[0, input_len:],
            skip_special_tokens=True
        ).strip()
        
        prediction = extract_label(full_response)
        
        results.append({
            "index": row.get("index", idx),
            "question": row["question"],
            "prediction": prediction,
            "raw_output": full_response
        })
        
    except Exception as e:
        tqdm.write(f"[{idx + 1}/{total}] ERROR: {e}")
        results.append({
            "index": row.get("index", idx),
            "question": row["question"],
            "prediction": "ERROR",
            "raw_output": str(e)
        })
    
    if (idx + 1) % 25 == 0:
        torch.cuda.empty_cache()

print("\n✅ Inference complete!")

In [ ]:
# ==================== SAVE RESULTS ====================
results_df = pd.DataFrame(results)

# Add ground truth if available
if "clarity_label" in eval_df.columns:
    results_df["clarity_label"] = eval_df["clarity_label"].values[:len(results_df)]

results_df.to_csv(OUTPUT_PATH, index=False)
print(f"✅ Saved {len(results_df)} predictions to {OUTPUT_PATH}")
results_df.head(10)

In [ ]:
# ==================== SUMMARY STATISTICS ====================
parse_errors = sum(1 for r in results if r['prediction'] == 'PARSE_ERROR')
skipped = sum(1 for r in results if r['prediction'] == 'SKIPPED')
errors = sum(1 for r in results if r['prediction'] == 'ERROR')
successful = len(results) - parse_errors - skipped - errors

print("\n" + "=" * 60)
print("SUMMARY")
print("=" * 60)
print(f"\nTotal:        {len(results)}")
print(f"Successful:   {successful}")
print(f"Parse errors: {parse_errors}")
print(f"Skipped:      {skipped}")
print(f"Errors:       {errors}")

print("\n📈 Prediction distribution:")
for label in ["Clear Reply", "Clear Non-Reply", "Ambivalent"]:
    count = sum(1 for r in results if r['prediction'] == label)
    pct = count / len(results) * 100 if results else 0
    print(f"   {label}: {count} ({pct:.1f}%)")

In [ ]:
# ==================== METRICS (if ground truth available) ====================
if "clarity_label" in results_df.columns:
    # Filter valid labels
    y_true = results_df["clarity_label"].astype(str).str.strip()
    y_pred = results_df["prediction"].str.strip()
    
    VALID = {"Clear Reply", "Clear Non-Reply", "Ambivalent"}
    mask = y_true.isin(VALID) & y_pred.isin(VALID)
    
    if mask.sum() > 0:
        y_true_f = y_true[mask]
        y_pred_f = y_pred[mask]
        
        print(f"\nValid for metrics: {mask.sum()} / {len(mask)}")
        
        accuracy = accuracy_score(y_true_f, y_pred_f)
        precision, recall, f1, _ = precision_recall_fscore_support(y_true_f, y_pred_f, average="macro")
        f1_w = f1_score(y_true_f, y_pred_f, average="weighted")
        
        print("\n" + "=" * 60)
        print("METRICS")
        print("=" * 60)
        print(f"Accuracy:         {accuracy:.4f}")
        print(f"Precision (Mac):  {precision:.4f}")
        print(f"Recall (Mac):     {recall:.4f}")
        print(f"F1 (Macro):       {f1:.4f}")
        print(f"F1 (Weighted):    {f1_w:.4f}")
        
        print("\n" + classification_report(y_true_f, y_pred_f, digits=4))
    else:
        print("\n⚠️  No valid predictions for metrics calculation")
else:
    print("\n⚠️  No ground truth labels in CSV - skipping metrics")

In [ ]:
# ==================== SAMPLE OUTPUTS ====================
print("\n🔍 Sample predictions (first 5):")
print("-" * 80)
for i in range(min(5, len(results_df))):
    row = results_df.iloc[i]
    q = str(row['question'])[:60] + "..." if len(str(row['question'])) > 60 else row['question']
    raw = str(row['raw_output'])[:80] + "..." if len(str(row['raw_output'])) > 80 else row['raw_output']
    print(f"{i+1}. Prediction: {row['prediction']}")
    print(f"   Question: {q}")
    print(f"   Raw: {raw}")
    print()

In [ ]:
# ==================== CLEANUP ====================
del model
torch.cuda.empty_cache()
print("✅ GPU memory cleared")